In [1]:
from config import config
import pandapower as pp

import random
import os
import h5py

import pandas as pd
import numpy as np
import xarray as xr

import pandapower.topology as top
import networkx as nx

from sklearn.model_selection import train_test_split

#### Inspect File Structure

In [2]:
def find_files(path):    
    files = [f for f in os.listdir(path) if os.path.isfile(os.path.join(path, f))]
    if not files:
        raise FileNotFoundError("No files in directory.")
    return [os.path.join(path,file) for file in files]

random.seed(41)
files = find_files(config.input_dir)
file_path = random.choice(files)
file_path

'/dss/dssfs04/lwp-dss-0002/pn98cu/pn98cu-dss-0000/EliasH/PostPowerflow/113_N3253500E4301500_80638_1_32_PV100_HP100_EV100_VarTar0_CapPr0.h5'

In [3]:
def list_h5_structure(file_path):
    with h5py.File(file_path, 'r') as f:
        def visitor_func(name, obj):
            if isinstance(obj, h5py.Group):
                print(f"Group: {name}")
            # elif isinstance(obj, h5py.Dataset):
            #     print(f"Dataset: {name}")
        f.visititems(visitor_func)
                
    # with h5py.File(file_path, 'r') as f:
    #     f.visititems(visitor_func)

list_h5_structure(file_path)

Group: pwrflw
Group: pwrflw/input
Group: pwrflw/input/demand_post
Group: pwrflw/input/demand_pre
Group: pwrflw/output
Group: pwrflw/output/post
Group: pwrflw/output/post/demand_import
Group: pwrflw/output/post/line_loads
Group: pwrflw/output/post/vm
Group: pwrflw/output/pre
Group: pwrflw/output/pre/demand_import
Group: pwrflw/output/pre/line_loads
Group: pwrflw/output/pre/vm
Group: pwrflw/urbs_out
Group: pwrflw/urbs_out/MILP
Group: pwrflw/urbs_out/MILP/reactive
Group: raw_data
Group: raw_data/buildings
Group: raw_data/consumers
Group: raw_data/region
Group: raw_data/weather
Group: urbs_in
Group: urbs_in/buy_sell_price
Group: urbs_in/commodity
Group: urbs_in/demand
Group: urbs_in/eff_factor
Group: urbs_in/process
Group: urbs_in/process_commodity
Group: urbs_in/storage
Group: urbs_in/supim
Group: urbs_in/weather
Group: urbs_out
Group: urbs_out/MILP
Group: urbs_out/MILP/cap_pro
Group: urbs_out/MILP/cap_pro_new
Group: urbs_out/MILP/cap_sto_c
Group: urbs_out/MILP/cap_sto_c_new
Group: urbs_o

#### Helper Functions

In [4]:
def get_PV_peak_capacity_up_to(buildings, factor=0.59161):
    total_PV_cap = sum(r[0] for roof_group in buildings["roofs"] for r in roof_group)
    return total_PV_cap*factor

def create_PV_prod_up_to(supim, processes, fraction_of_total=0.6766):
        ### Maximum installable:
    pv_100_peak = processes[processes["Process"].str.startswith("Rooftop PV")].set_index(["Site", "Process"])
    pv_100_peak = pv_100_peak["cap-up"].to_frame().T
    cap_map = {}
    cap_cols = pv_100_peak.columns
    for site, cap_proc in cap_cols:
        # convert 'Rooftop PV_x_y' -> 'solar_x_y'
        solar_proc = cap_proc.replace('Rooftop PV', 'solar')
        cap_map[(site, solar_proc)] = pv_100_peak[(site, cap_proc)].iloc[0]

    # Step 2: Build a Series indexed by df_ts.columns with the matching cap-up values
    cap_series = pd.Series(
        [cap_map.get((site, proc), pd.NA) for site, proc in supim.columns],
        index=supim.columns
    )

    # Step 3: Multiply each timeseries column by its cap-up value
    df_scaled = supim * cap_series

    max_energy = df_scaled.sum().sum()
    target_energy = max_energy * fraction_of_total

    
    supim_sums = supim.sum(axis=0)       # total capacity factor per column
    energy_sums = df_scaled.sum(axis=0)  # total energy per column

    # 2. Sort by capacity factor sum
    order = supim_sums.sort_values(ascending=False).index

    # 3. Greedy accumulation
    selected = []
    selected_ts = []
    total_energy = 0.0

    for col in order:
        col_energy = energy_sums[col]
        if total_energy + col_energy <= target_energy:
            # take whole column
            selected.append((col, 1.0))
            selected_ts.append(df_scaled[col])
            total_energy += col_energy
        else:
            # take only a fraction of this column
            remaining = target_energy - total_energy
            frac = remaining / col_energy
            selected.append((col, frac))
            selected_ts.append(df_scaled[col] * frac)
            total_energy = target_energy
            break

    production_ts = pd.concat(selected_ts, axis=1).sum(axis=1)

    ### Actually installed:
    return production_ts
    

In [5]:
def get_PV_energy_capacity(roofs, supim):
    supim = pd.DataFrame(supim.sum(axis=0))
    supim.rename(columns={0:"cap"}, inplace=True)

    roofs = roofs.explode("roofs")
    roofs["pro"] = roofs["roofs"].apply(lambda x: f"solar_{x[1]}_{x[2]}")
    roofs["cap"] = roofs["roofs"].apply(lambda x: x[0])
    roofs.drop(columns=["roofs"], inplace=True)
    roofs.rename(columns={"bus":"sit"}, inplace=True)
    roofs.set_index(["sit", "pro"], inplace=True)

    return (roofs*supim).sum().values[0]

def get_mobility_capacity(df_storage):
    return df_storage[df_storage["Storage"].str.startswith("mobility_storage")]["inst-cap-c"].sum() 

def get_number_lines(grid):
    trafo_buses = grid.trafo[["hv_bus", "lv_bus"]].values[0]
    ext_grid_bus = int(grid.ext_grid.loc[0, "bus"]) # bus which is the external import bus
    lv_bus = [bus for bus in trafo_buses if bus!=ext_grid_bus][0]

    # Retrieve first section of household cable strands
    hh_strands = grid.line[grid.line["from_bus"]==lv_bus].index.values     # retrieve all household strands
    return int(len(hh_strands))

def get_total_length_grid(grid):
    return grid.line["length_km"].sum()

def get_total_resistance_grid(grid):
    return (grid.line["length_km"]*grid.line["r_ohm_per_km"]).sum()

In [6]:
import numpy as np
import pandas as pd

def get_PV_100(supim, processes):
    cols = supim.columns

    # Identify solar processes in supim (process in level 1)
    pro_vals = cols.get_level_values(1).astype(str)
    solar_mask = pro_vals.str.startswith("solar_")
    solar_cols = cols[solar_mask]

    # Initialize per-column factor with 1.0 so non-solar stay unchanged
    cap_factor = pd.Series(1.0, index=cols, dtype=float)
    for sit, pro in solar_cols:
        suffix = pro.split("solar_", 1)[1]
        mapped_proc = f"Rooftop PV_{suffix}"
        cap = processes[(processes["Site"] == sit) & (processes["Process"] == str(mapped_proc))].loc[:, "cap-up"].values[0]
        cap_factor[(sit, pro)] = float(cap)

    # Multiply each column's time series by its cap factor
    supim_cap = supim.multiply(cap_factor, axis=1)
    solar_100 = supim_cap.sum(axis=1)
    return solar_100


def get_heat_and_cop(eff_factor, demand):
    # Extract space and water heating demands (sum over all sites/processes per timestep)
    mask = demand.columns.get_level_values(1).str.startswith("space_heat")
    space_heat = demand.loc[:, mask].droplevel(1, axis=1)
    space_heat_tot = space_heat.sum(axis=1)

    mask = demand.columns.get_level_values(1).str.startswith("water_heat")
    water_heat = demand.loc[:, mask].droplevel(1, axis=1)
    water_heat_tot = water_heat.sum(axis=1)

    # Extract heat pump COPs
    mask = eff_factor.columns.get_level_values(1).str.startswith("heatpump_air")
    hp_cop = eff_factor.loc[:, mask].droplevel(1, axis=1)

    # Weighted average COP using heat demand as weights
    total_heat_cols = (water_heat + space_heat)  # per-column weights
    denom = water_heat_tot + space_heat_tot      # per-timestep total weight
    weighted_sum = hp_cop.multiply(total_heat_cols).sum(axis=1)

    # Compute weighted average where denom > 0
    weighted_cop = pd.Series(np.nan, index=denom.index, dtype=float)
    nonzero = denom != 0
    weighted_cop.loc[nonzero] = (weighted_sum.loc[nonzero] / denom.loc[nonzero])

    # Fallback: for timesteps with zero total heat demand, use simple mean COP across columns
    row_mean_cop = hp_cop.mean(axis=1)
    weighted_cop.loc[~nonzero] = row_mean_cop.loc[~nonzero]

    return space_heat_tot, water_heat_tot, weighted_cop

In [7]:
def analyze_feeders(net, trafo_idx=0):
    """
    For each feeder emanating from the LV-side of transformer trafo_idx in net,
    compute total length (km), total R (Ω), number of consumer buses,
    list of feeder-internal line segments, and list of trunk line segments
    (from LV bus to first bus of each feeder).

    Returns a dict keyed by feeder number.
    """

    # 1. Identify the LV bus of the transformer
    lv_bus = net.trafo.lv_bus.at[trafo_idx]

    # 2. Build an undirected graph of the network (buses + lines)
    G = top.create_nxgraph(net, include_trafos=False, respect_switches=False)

    # 3. Remove the trafo LV bus to split the feeders
    G_without_trafo = G.copy()
    G_without_trafo.remove_node(lv_bus)

    # 4. Find connected components (each is one feeder)
    components = list(nx.connected_components(G_without_trafo))

    feeders = {}
    for i, comp in enumerate(components, start=1):
        # 4a. Feeder-internal lines: both ends in this component
        mask_internal = net.line.apply(
            lambda row: (row.from_bus in comp) and (row.to_bus in comp),
            axis=1
        )
        feeder_segments = net.line.index[mask_internal].tolist()

        # 4b. Trunk segments: those connecting LV bus to any node in comp
        mask_trunk = net.line.apply(
            lambda row: ((row.from_bus == lv_bus and row.to_bus in comp) or
                         (row.to_bus == lv_bus and row.from_bus in comp)),
            axis=1
        )
        trunk_segments = net.line.index[mask_trunk].tolist()

        # 4c. Combine for totals if needed
        all_segments = feeder_segments + trunk_segments

        total_length = net.line.loc[all_segments, "length_km"].sum()
        total_R = (net.line.loc[all_segments, "length_km"] *
                   net.line.loc[all_segments, "r_ohm_per_km"]).sum()

        # 4d. Consumer buses: buses in comp with at least one load
        load_buses = set(net.load.bus.loc[net.load.bus.isin(comp)])
        num_consumer_buses = len(load_buses)

        feeders[f"Feeder_{i}"] = {
            "buses_in_feeder": sorted(comp),
            "trunk_segments": trunk_segments,
            "feeder_segments": feeder_segments,
            "total_length_km": total_length,
            "total_R_ohm": total_R,
            "num_consumer_buses": num_consumer_buses
        }

    return feeders

def get_feeder_statistics(net, mean_line_powers):
    list_P_R = []
    list_nbldg_R = []
    list_R = []
    list_L = []

    for i, feeder in analyze_feeders(net).items():
        if feeder["trunk_segments"] != []:
            P = mean_line_powers[feeder["trunk_segments"]]
            R = feeder["total_R_ohm"]
            L = feeder["total_length_km"]
            n_bldng = feeder["num_consumer_buses"]

            list_P_R.append(P*R)
            list_nbldg_R.append(n_bldng*R)
            list_R.append(R)
            list_L.append(L)

    feeder_stats = {
        "max_line_nbldg_x_R": np.max(list_nbldg_R),
        "max_line_P_x_R": np.max(list_P_R),
        "max_line_R": np.max(list_R),
        "max_line_L": np.max(list_L)
    }

    return feeder_stats

In [8]:
# def draw_grid(grid):
#     """ Creates an image of the network graph
#         Args: net.line dataframe from pandapower network
#     """
#     df_lines = grid.line
#     # Create an undirected graph
#     G = nx.Graph()

#     # Add edges to the graph
#     for _, row in df_lines.iterrows():
#         G.add_edge(row["from_bus"], row["to_bus"])

#     pos = nx.spring_layout(G, k=0.05, iterations=120)
#     # Draw the graph
#     plt.figure(figsize=(5, 4))
#     nx.draw(G, pos, with_labels=True, node_color='skyblue', node_size=25, font_size=6, font_weight='bold', edge_color='gray')
#     plt.title("Network Graph of Nodes")
#     plt.show()

# draw_grid(net)

#### 1. Readout all data and concatenate from all files

In [9]:
### Prepare storage arrays with correct dimensions
# X timeseries input
batch_size = len(files)
X_ts_names = ["demand_net_active_pre","demand_net_reactive_pre", 
              "T", "GHI", "DNI", "DHI", "PV_prod_100", "PV_prod_expected", 
              "heat_water", "heat_space", "cop_avg", "mob_avail", "mob_last_avail", "mobility",
              "PV_prod_true", "heat_true", "mobility_true"  # these are the actual urbs results, should never be used apart from attribution of losses
              ]
X_ts_name2index = {name: i for i,name in enumerate(X_ts_names)}
X_n_ts = len(X_ts_names)
hours = 8760

coords = {
    "batch": np.arange(batch_size),
    "timeseries": X_ts_names,
    "hour": np.arange(hours)}

ds_X_ts = xr.Dataset(   
    {"ts_val": (("batch", "timeseries", "hour"), np.zeros((batch_size, X_n_ts,hours), dtype=np.float32))},
    coords=coords)


# y timeseries target output
y_ts_names = ["demand_net_active_post", "demand_net_reactive_post", "demand_net_active_post_nopwrflw", "demand_net_reactive_post_nopwrflw"]
y_ts_name2index = {name: i for i,name in enumerate(y_ts_names)}
y_n_ts = len(y_ts_names)

coords = {
    "batch": np.arange(batch_size),
    "timeseries": y_ts_names,
    "hour": np.arange(hours)}

ds_y_ts = xr.Dataset(   
    {"ts_val": (("batch", "timeseries", "hour"), np.zeros((batch_size, y_n_ts,hours), dtype=np.float32))},
    coords=coords)

# X scalar input
X_sclr_names = ["n_bldng","n_flats","n_occ","bldng_area_base_sum","res_bldng_area_base_sum", "nonres_bldng_area_base_sum","bldng_area_floors_sum",
                "n_radiator", "n_res_buildings", "n_nonres_buildings",
                "PV_cap_power_max_sum","PV_cap_power_exp_sum","PV_cap_energy_max_sum",
                "regiostar7", "n_cars", "car_battery_cap",
                "n_lines","tot_len_grid","tot_R_grid",
                "max_line_nbldg_x_R","max_line_P_x_R","max_line_R","max_line_L"]
X_sclr_name2index = {name: i for i,name in enumerate(X_sclr_names)}
X_n_sclr = len(X_sclr_names)

coords = coords = {
    "batch": np.arange(batch_size),
    "scalar": X_sclr_names}

ds_X_sclr = xr.Dataset(   
    {"sclr_val": (("batch", "scalar"), np.zeros((batch_size,X_n_sclr), dtype=np.float32))},
    coords=coords)


# y scalar target output
y_sclr_names = ["demand_net_active_agg_post","consumption_net_active_agg_post","feedin_net_active_agg_post",
                "demand_net_reactive_agg_post","consumption_net_active_peak_post","feedin_net_active_peak_post",
                "voltage_violation_lower_min","voltage_violation_upper_max"]
y_sclr_name2index = {name: i for i,name in enumerate(y_sclr_names)}
y_n_sclr = len(y_sclr_names)

coords = coords = {
    "batch": np.arange(batch_size),
    "scalar": y_sclr_names}

ds_y_sclr = xr.Dataset(   
    {"sclr_val": (("batch", "scalar"), np.zeros((batch_size,y_n_sclr), dtype=np.float32))},
    coords=coords)

In [10]:
### Loop over all files and read-out wanted inputs
for i, file in enumerate(files):
    if i % 100 == 0 and i>0: print(f"So far processed {i} files...")
    with pd.HDFStore(file) as store:
        # X timeseries input:
        j = X_ts_name2index["demand_net_active_pre"]
        ds_X_ts["ts_val"].data[i,j,:] = store["pwrflw/output/pre/demand_import"]["p_mw"].values

        j = X_ts_name2index["demand_net_reactive_pre"]
        ds_X_ts["ts_val"].data[i,j,:] = store["pwrflw/output/pre/demand_import"]["q_mvar"].values

        j = X_ts_name2index["T"]
        ds_X_ts["ts_val"].data[i,j,:] = store["raw_data/weather"]["temp_air"].values

        j = X_ts_name2index["GHI"]
        ds_X_ts["ts_val"].data[i,j,:] = store["raw_data/weather"]["ghi"].values

        j = X_ts_name2index["DNI"]
        ds_X_ts["ts_val"].data[i,j,:] = store["raw_data/weather"]["dni"].values

        j = X_ts_name2index["DHI"]
        ds_X_ts["ts_val"].data[i,j,:] = store["raw_data/weather"]["dhi"].values

        supim = store["urbs_in/supim"]
        processes = store["urbs_in/process"]
        demand = store["urbs_in/demand"]
        eff_factor = store["urbs_in/eff_factor"]
        e_pro_out = store["urbs_out/MILP/e_pro_out"]
        e_pro_in = store["urbs_out/MILP/e_pro_in"]

        mask = e_pro_out.index.get_level_values("pro").str.startswith("Rooftop PV")
        PV_prod_true = e_pro_out[mask]
        PV_prod_true = PV_prod_true.reset_index()
        PV_prod_true = PV_prod_true.pivot_table(
            index="t",
            columns=["sit", "pro"],
            values=PV_prod_true.columns.difference(["t", "stf", "sit", "pro", "com"])[0])
        j = X_ts_name2index["PV_prod_true"]
        ds_X_ts["ts_val"].data[i,j,:] = PV_prod_true.sum(axis=1).values

        j = X_ts_name2index["PV_prod_expected"]
        ds_X_ts["ts_val"].data[i,j,:] = create_PV_prod_up_to(supim.copy(), processes.copy()).values

        j = X_ts_name2index["PV_prod_100"]
        ds_X_ts["ts_val"].data[i,j,:] = get_PV_100(supim.copy(), processes.copy()).values

        heat_space, heat_water, cop_avg = get_heat_and_cop(eff_factor.copy(), demand.copy())
        j = X_ts_name2index["heat_space"]
        ds_X_ts["ts_val"].data[i,j,:] = heat_space.values
        j = X_ts_name2index["heat_water"]
        ds_X_ts["ts_val"].data[i,j,:] = heat_water.values
        j = X_ts_name2index["cop_avg"]
        ds_X_ts["ts_val"].data[i,j,:] = cop_avg.values

        mask = e_pro_in.index.get_level_values("pro").str.startswith("heatpump_air")
        heat_dem_HP_true = e_pro_in[mask]
        heat_dem_HP_true = heat_dem_HP_true.reset_index()
        heat_dem_HP_true = heat_dem_HP_true.pivot_table(
            index="t",
            columns=["sit", "pro"],
            values=heat_dem_HP_true.columns.difference(["t", "stf", "sit", "pro", "com"])[0])
        heat_dem_HP_true = heat_dem_HP_true.sum(axis=1)
        mask = e_pro_in.index.get_level_values("pro").str.startswith("heatpump_booster")
        heat_dem_bst_true = e_pro_in[mask]
        heat_dem_bst_true = heat_dem_bst_true.reset_index()
        heat_dem_bst_true = heat_dem_bst_true.pivot_table(
            index="t",
            columns=["sit", "pro"],
            values=heat_dem_bst_true.columns.difference(["t", "stf", "sit", "pro", "com"])[0])
        heat_dem_bst_true = heat_dem_bst_true.sum(axis=1)
        j = X_ts_name2index["heat_true"]
        ds_X_ts["ts_val"].data[i,j,:] = (heat_dem_HP_true.add(heat_dem_bst_true, fill_value=0)).values

        mask = eff_factor.columns.get_level_values(1).str.startswith("charging_station")
        mobility = eff_factor.loc[:, mask]
        mobility_avail = mobility.sum(axis=1)
        j = X_ts_name2index["mob_avail"]
        ds_X_ts["ts_val"].data[i,j,:] = mobility_avail.values

        mask_last = (mobility == 1) & (mobility.shift(-1) == 0)
        mobility_last_avail = mobility.where(mask_last).sum(axis=1)
        j = X_ts_name2index["mob_last_avail"]
        ds_X_ts["ts_val"].data[i,j,:] = mobility_last_avail.values

        try:
            mask = demand.columns.get_level_values(1).str.startswith("mobility")
            j = X_ts_name2index["mobility"]
            ds_X_ts["ts_val"].data[i,j,:] = demand.loc[:,mask].sum(axis=1).values
        except KeyError:
            pass

        try:
            mask = e_pro_in.index.get_level_values("pro").str.startswith("charging_station")
            mob_dem_true = e_pro_in[mask]
            mob_dem_true = mob_dem_true.reset_index()
            mob_dem_true = mob_dem_true.pivot_table(
                index="t",
                columns=["sit", "pro"],
                values=mob_dem_true.columns.difference(["t", "stf", "sit", "pro", "com"])[0])
            j = X_ts_name2index["mobility_true"]
            ds_X_ts["ts_val"].data[i,j,:] = mob_dem_true.sum(axis=1).values
        except (ValueError, KeyError):
            pass

        # y timeseries target
        j = y_ts_name2index["demand_net_active_post"]
        ds_y_ts["ts_val"].data[i,j,:] = store["pwrflw/output/post/demand_import"]["p_mw"].values

        j = y_ts_name2index["demand_net_reactive_post"]
        ds_y_ts["ts_val"].data[i,j,:] = store["pwrflw/output/post/demand_import"]["q_mvar"].values

        j = y_ts_name2index["demand_net_active_post_nopwrflw"]
        ds_y_ts["ts_val"].data[i,j,:] = store["pwrflw/input/demand_post"].xs('electricity', axis=1, level=1, drop_level=False).sum(axis=1)

        j = y_ts_name2index["demand_net_reactive_post_nopwrflw"]
        ds_y_ts["ts_val"].data[i,j,:] = store["pwrflw/input/demand_post"].xs('electricity-reactive', axis=1, level=1, drop_level=False).sum(axis=1)


        # X scalar input
        j = X_sclr_name2index["n_bldng"]
        ds_X_sclr["sclr_val"].data[i,j] = len(store["raw_data/buildings"])

        j = X_sclr_name2index["n_flats"]
        ds_X_sclr["sclr_val"].data[i,j] = int(store["raw_data/buildings"]["houses_per_building"].sum())

        j = X_sclr_name2index["n_occ"]
        ds_X_sclr["sclr_val"].data[i,j] = int(round(sum(store["raw_data/buildings"]["occ_list"].sum()),0))

        j = X_sclr_name2index["bldng_area_base_sum"]
        ds_X_sclr["sclr_val"].data[i,j] = store["raw_data/buildings"]["area"].sum()

        j = X_sclr_name2index["bldng_area_floors_sum"]
        ds_X_sclr["sclr_val"].data[i,j] = (store["raw_data/buildings"]["area"]*store["raw_data/buildings"]["floors"]).sum()

        buildings = store["raw_data/buildings"]
        j = X_sclr_name2index["n_res_buildings"]
        ds_X_sclr["sclr_val"].data[i,j] = len(buildings[buildings["use"]=="Residential"])

        j = X_sclr_name2index["n_nonres_buildings"]
        ds_X_sclr["sclr_val"].data[i,j] = len(buildings[buildings["use"]!="Residential"])

        j = X_sclr_name2index["res_bldng_area_base_sum"]
        ds_X_sclr["sclr_val"].data[i,j] = buildings[buildings["use"]=="Residential"]["area"].sum()

        j = X_sclr_name2index["nonres_bldng_area_base_sum"]
        ds_X_sclr["sclr_val"].data[i,j] = buildings[buildings["use"]!="Residential"]["area"].sum()
        
        j = X_sclr_name2index["n_radiator"]
        ds_X_sclr["sclr_val"].data[i,j] = len(buildings[buildings["heating_type"]=="radiator"])

        j = X_sclr_name2index["PV_cap_power_max_sum"]
        ds_X_sclr["sclr_val"].data[i,j] = sum([roofs[0] for buildings in store["raw_data/buildings"]["roofs"].values for roofs in buildings])

        j = X_sclr_name2index["PV_cap_power_exp_sum"]
        ds_X_sclr["sclr_val"].data[i,j] = get_PV_peak_capacity_up_to(store["raw_data/buildings"])

        j = X_sclr_name2index["PV_cap_energy_max_sum"]
        ds_X_sclr["sclr_val"].data[i,j] = get_PV_energy_capacity(store["raw_data/buildings"][["bus","roofs"]], store["urbs_in/supim"])

        j = X_sclr_name2index["regiostar7"]
        ds_X_sclr["sclr_val"].data[i,j] = store["raw_data/region"]["regio7"]

        j = X_sclr_name2index["n_cars"]
        ds_X_sclr["sclr_val"].data[i,j] = int(store["raw_data/buildings"]["n_cars_tot"].sum())

        j = X_sclr_name2index["car_battery_cap"]
        ds_X_sclr["sclr_val"].data[i,j] = get_mobility_capacity(store["urbs_in/storage"])

        # y scalar target
        ["consumption_net_active_peak_post","feedin_net_active_peak_post","voltage_violation_lower_min","voltage_violation_upper_max"]
        j = y_sclr_name2index["demand_net_active_agg_post"]
        ds_y_sclr["sclr_val"].data[i,j] = store["pwrflw/output/post/demand_import"]["p_mw"].sum()

        j = y_sclr_name2index["consumption_net_active_agg_post"]
        ds_y_sclr["sclr_val"].data[i,j] = store["pwrflw/output/post/demand_import"]["p_mw"].clip(lower=0).sum()

        j = y_sclr_name2index["feedin_net_active_agg_post"]
        ds_y_sclr["sclr_val"].data[i,j] = store["pwrflw/output/post/demand_import"]["p_mw"].clip(upper=0).sum()

        j = y_sclr_name2index["demand_net_reactive_agg_post"]
        ds_y_sclr["sclr_val"].data[i,j] = store["pwrflw/output/post/demand_import"]["q_mvar"].sum()

        j = y_sclr_name2index["consumption_net_active_peak_post"]
        ds_y_sclr["sclr_val"].data[i,j] = store["pwrflw/output/post/demand_import"]["p_mw"].max()

        j = y_sclr_name2index["feedin_net_active_peak_post"]
        ds_y_sclr["sclr_val"].data[i,j] = store["pwrflw/output/post/demand_import"]["p_mw"].min()

        j = y_sclr_name2index["voltage_violation_lower_min"]
        ds_y_sclr["sclr_val"].data[i,j] = store["pwrflw/output/post/vm"].min().min()

        j = y_sclr_name2index["voltage_violation_upper_max"]
        ds_y_sclr["sclr_val"].data[i,j] = store["pwrflw/output/post/vm"].max().max()

        # Extract line loads for following analysis
        line_loads = store["pwrflw/output/pre/line_loads"]
        line_p_flow = line_loads.loc[:, line_loads.columns.get_level_values(1) == 'p_from_mw'].mean(axis=0)

    # Extra X scalar input data which needs different readout
    with h5py.File(file, 'r') as file:
        net = pp.from_json_string(file["raw_data/net"][()])
        line_stats = get_feeder_statistics(net, line_p_flow)

        j = X_sclr_name2index["n_lines"]
        ds_X_sclr["sclr_val"].data[i,j] = get_number_lines(net)

        j = X_sclr_name2index["tot_len_grid"]
        ds_X_sclr["sclr_val"].data[i,j] = get_total_length_grid(net)

        j = X_sclr_name2index["tot_R_grid"]
        ds_X_sclr["sclr_val"].data[i,j] = get_total_resistance_grid(net)

        j = X_sclr_name2index["max_line_nbldg_x_R"]
        ds_X_sclr["sclr_val"].data[i,j] = line_stats["max_line_nbldg_x_R"]

        j = X_sclr_name2index["max_line_P_x_R"]
        ds_X_sclr["sclr_val"].data[i,j] = line_stats["max_line_P_x_R"]

        j = X_sclr_name2index["max_line_R"]
        ds_X_sclr["sclr_val"].data[i,j] = line_stats["max_line_R"]

        j = X_sclr_name2index["max_line_L"]
        ds_X_sclr["sclr_val"].data[i,j] = line_stats["max_line_L"]

So far processed 100 files...
So far processed 200 files...
So far processed 300 files...
So far processed 400 files...
So far processed 500 files...
So far processed 600 files...
So far processed 700 files...
So far processed 800 files...
So far processed 900 files...
So far processed 1000 files...
So far processed 1100 files...


In [ ]:
import xarray as xr

FILEPATH = "Data/all_data.h5"

ds_X_ts.to_netcdf(
    path=FILEPATH,
    mode="w",               # create or overwrite
    format="NETCDF4",       # HDF5-based netCDF4 format
    engine="h5netcdf",      # h5netcdf backend
    group="X_ts"            # subgroup name (no leading slash)
)

ds_y_ts.to_netcdf(
    path=FILEPATH,
    mode="a",               # append, do NOT overwrite
    format="NETCDF4",       # same format
    engine="h5netcdf",      # same engine
    group="y_ts"
)

ds_X_sclr.to_netcdf(
    path=FILEPATH,
    mode="a",
    format="NETCDF4",
    engine="h5netcdf",
    group="X_sclr"
)

ds_y_sclr.to_netcdf(
    path=FILEPATH,
    mode="a",
    format="NETCDF4",
    engine="h5netcdf",
    group="y_sclr"
)

### 2. Train-Test Split + Conversion To Time Series

In [13]:
### Readout all inputs
FILEPATH = "Data/all_data.h5"
# Read the dataset from group "X_ts"
ds_X_ts = xr.open_dataset(
    FILEPATH,
    group="X_ts",
    engine="h5netcdf")
# Read the dataset from group "y_ts"
ds_y_ts = xr.open_dataset(
    FILEPATH,
    group="y_ts",
    engine="h5netcdf")
# Read the dataset from group "X_sclr"
ds_X_sclr = xr.open_dataset(
    FILEPATH,
    group="X_sclr",
    engine="h5netcdf")
# Read the dataset from group "y_sclr"
ds_y_sclr = xr.open_dataset(
    FILEPATH,
    group="y_sclr",
    engine="h5netcdf")

In [14]:
#### Train-Test-Split
seed=42

n_batch = ds_X_ts.sizes['batch']
batch_idxs = np.arange(n_batch)
train_idx, test_idx = train_test_split(
    batch_idxs,
    test_size=1/6,      # 200/1200 for test
    random_state=seed,
    shuffle=True)

ds_X_ts_train = ds_X_ts.isel(batch=train_idx)
ds_X_ts_test  = ds_X_ts.isel(batch=test_idx)

ds_y_ts_train = ds_y_ts.isel(batch=train_idx)
ds_y_ts_test  = ds_y_ts.isel(batch=test_idx)

ds_X_sclr_train = ds_X_sclr.isel(batch=train_idx)
ds_X_sclr_test  = ds_X_sclr.isel(batch=test_idx)

ds_y_sclr_train = ds_y_sclr.isel(batch=train_idx)
ds_y_sclr_test  = ds_y_sclr.isel(batch=test_idx)

In [13]:
# ### Readout all inputs
# FILEPATH = "../../0_preprocessing/Data/train.h5"
# # Read the dataset from group "X_ts"
# ds_X_ts = xr.open_dataset(
#     FILEPATH,
#     group="X_ts",
#     engine="h5netcdf")
# # Read the dataset from group "y_ts"
# ds_y_ts = xr.open_dataset(
#     FILEPATH,
#     group="y_ts",
#     engine="h5netcdf")
# # Read the dataset from group "X_sclr"
# ds_X_sclr = xr.open_dataset(
#     FILEPATH,
#     group="X_sclr",
#     engine="h5netcdf")
# # Read the dataset from group "y_sclr"
# ds_y_sclr = xr.open_dataset(
#     FILEPATH,
#     group="y_sclr",
#     engine="h5netcdf")

In [15]:
def create_timeseries_df(ds_X_ts, ds_X_sclr, ds_y_ts):
    ### Convert X timeseries to DataFrame
    df_long = ds_X_ts["ts_val"].to_dataframe()
    df_X_ts = df_long.reset_index().pivot_table(
        index=['batch', 'hour'],           # the row index levels
        columns='timeseries',            # the former series‑dimension
        values='ts_val',                     # the cell contents
        aggfunc='last'
    )
    df_X_ts.index.names = ['batch', 'hour']
    df_X_ts.columns.name = None  # drop the name ‘n_timeseries’

    ### Convert X scalar to DataFrame
    df_long = ds_X_sclr["sclr_val"].to_dataframe()
    df_X_sclr = df_long.reset_index().pivot(
        index=['batch'],           # the row index levels
        columns='scalar',            # the former series‑dimension
        values='sclr_val'                     # the cell contents
    )
    df_X_sclr.index.names = ['batch']
    df_X_sclr.columns.name = None  # drop the name ‘n_timeseries’

    ### Combine:
    df_X = df_X_ts.join(df_X_sclr, on='batch')

    ### Convert y timeseries to DataFrame
    df_long = ds_y_ts["ts_val"].to_dataframe()
    df_y = df_long.reset_index().pivot(
        index=['batch', 'hour'],           # the row index levels
        columns='timeseries',            # the former series‑dimension
        values='ts_val'                     # the cell contents
    )
    df_y.index.names = ['batch', 'hour']
    df_y.columns.name = None  # drop the name ‘n_timeseries’

    return df_X, df_y

In [16]:
df_X_train, df_y_train = create_timeseries_df(ds_X_ts_train, ds_X_sclr_train, ds_y_ts_train)
df_X_train.to_hdf("Data/ts_train.h5", key="X", mode="w")
df_y_train.to_hdf("Data/ts_train.h5", key="y", mode="a")

df_X_test, df_y_test = create_timeseries_df(ds_X_ts_test, ds_X_sclr_test, ds_y_ts_test)
df_X_test.to_hdf("Data/ts_test.h5", key="X", mode="w")
df_y_test.to_hdf("Data/ts_test.h5", key="y", mode="a")

### (3. Only for small grid approach of thesis)

In [ ]:
df_train_X = pd.read_hdf("Data/ts_train.h5", key="X")
df_train_y = pd.read_hdf("Data/ts_train.h5", key="y")
df_test_X = pd.read_hdf("Data/ts_test.h5", key="X")
df_test_y = pd.read_hdf("Data/ts_test.h5", key="y")

def _filter_small_batches(df_X, df_y, thresh=4):
    # Get per-batch building counts (scalars are constant across hours)
    bld_counts = (
        df_X.groupby(level="batch")[["n_res_buildings", "n_nonres_buildings"]]
            .first()
            .sum(axis=1)
    )
    keep_batches = bld_counts[bld_counts > thresh].index
    df_X_f = df_X.loc[keep_batches]
    df_y_f = df_y.loc[keep_batches]
    return df_X_f, df_y_f

df_train_X, df_train_y = _filter_small_batches(df_train_X, df_train_y)
df_test_X, df_test_y   = _filter_small_batches(df_test_X, df_test_y)

with pd.HDFStore("Data/ts_train_large_grids.h5", mode="w") as store:
    store["X"] = df_train_X
    store["y"] = df_train_y
with pd.HDFStore("Data/ts_test_large_grids.h5", mode="w") as store:
    store["X"] = df_test_X
    store["y"] = df_test_y